In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import ResNet50_Weights

In [ ]:
resnet50 = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
print(resnet50)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 76.6MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
def load_pretrained_resnet(class_num):
    resnet50 = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)  # 혹은 models.resnet50(pretrained=True)
    # change classifier
    resnet50.fc = nn.Linear(in_features=2048, out_features=class_num, bias=True)
    # freeze classifier
    for name, p in resnet50.named_parameters():
        if 'fc' in name:
            p.requires_grad = True
        else:
            p.requires_grad = False
    return resnet50

In [ ]:
model = load_pretrained_resnet(10)

In [ ]:
for name, p in model.named_parameters():
  print(f"param: {name} \t requires_grad: {p.requires_grad}")

In [ ]:
model()

### model inference result

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

In [ ]:
def load_model(model_path, num_classes):
    model = models.resnet50()  # 모델 구조 초기화 : 기본 ResNet50 모델
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # 출력 계층 수정
    model.load_state_dict(torch.load(model_path))  # 가중치 불러오기
    model.eval()  # 추론 모드로 설정
    return model

In [ ]:
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # 배치 차원 추가
    return image

In [ ]:
def predict(model, processed_image):
    with torch.no_grad():  # 그라디언트 계산 비활성화
        outputs = model(processed_image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()  # 클래스 인덱스 반환

In [ ]:
def main(model_path, num_classes, image_path):
    model = load_model(model_path, num_classes)
    image = preprocess_image(image_path)
    prediction = predict(model, image)
    print("Predicted class index:", prediction)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
TRAINED_PATH = '/content/drive/MyDrive/cnn/weights/resnet_finetuned.pth'
TEST_IMG_PATH = '/content/drive/MyDrive/cnn/test_images/image_rotten.jpg'

main(TRAINED_PATH, 2, TEST_IMG_PATH)

Predicted class index: 0


In [17]:
main()